In [ ]:
"""
{
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "vs:KommunerNyeste",
        "values": [
          "*"
        ]
      }
    },
    {
      "code": "Alder",
      "selection": {
        "filter": "vs:AlleAldre00B",
        "values": []
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2018"
        ]
      }
    }
  ],
  "response": {
    "format": "csv"
  }
}
"""

In [1]:
import requests

## Earnings

In [3]:
URL = "https://data.ssb.no/api/v0/no/table/11418/"

query = {
  "query": [
    {
      "code": "MaaleMetode",
      "selection": {
        "filter": "item",
        "values": [
          "02",
          "01"
        ]
      }
    },
    {
      "code": "Yrke",
      "selection": {
        "filter": "all",
        "values": [
         "*"
        ]
      }
    },
    {
      "code": "Sektor",
      "selection": {
        "filter": "item",
        "values": [
          "ALLE"
        ]
      }
    },
    {
      "code": "Kjonn",
      "selection": {
        "filter": "item",
        "values": [
          "0"
        ]
      }
    },
    {
      "code": "AvtaltVanlig",
      "selection": {
        "filter": "item",
        "values": [
          "0"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "Manedslonn"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2019"
        ]
      }
    }
  ],
  "response": {
    "format": "csv"
  }
}

In [8]:
response = requests.post(URL, json=query)

In [9]:
response.status_code

200

In [10]:
response.text

'"statistikkmål","yrke","sektor","kjønn","avtalt/vanlig arbeidstid per uke","Månedslønn (kr) 2019"\r\n"Gjennomsnitt","0-9 Alle yrker","Sum alle sektorer","Begge kjønn","I alt",47290\r\n"Gjennomsnitt","00 Uoppgitt / yrker som ikke kan identifiseres","Sum alle sektorer","Begge kjønn","I alt",:\r\n"Gjennomsnitt","000 Uoppgitt / yrker som ikke kan identifiseres","Sum alle sektorer","Begge kjønn","I alt",:\r\n"Gjennomsnitt","0000 Uoppgitt / yrker som ikke kan identifiseres","Sum alle sektorer","Begge kjønn","I alt",:\r\n"Gjennomsnitt","01 Offiserer fra fenrik og høyere grad","Sum alle sektorer","Begge kjønn","I alt",63990\r\n"Gjennomsnitt","011 Offiserer fra fenrik og høyere grad","Sum alle sektorer","Begge kjønn","I alt",63990\r\n"Gjennomsnitt","0110 Offiserer fra fenrik og høyere grad","Sum alle sektorer","Begge kjønn","I alt",63990\r\n"Gjennomsnitt","02 Befal med sersjant grad","Sum alle sektorer","Begge kjønn","I alt",46810\r\n"Gjennomsnitt","021 Befal med sersjant grad","Sum alle sekto

In [11]:
len(response.text)

109645

## Sysselsetting kommune

In [12]:
URL = "https://data.ssb.no/api/v0/no/table/11619"

query = {
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "all",
        "values": [
          "*"
        ]
      }
    },
    {
      "code": "Kjonn",
      "selection": {
        "filter": "item",
        "values": [
          "0"
        ]
      }
    },
    {
      "code": "Alder",
      "selection": {
        "filter": "item",
        "values": [
          "15-74"
        ]
      }
    },
    {
      "code": "Yrke",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2",
          "3_01-03",
          "4",
          "5",
          "6",
          "7",
          "8",
          "9"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2018"
        ]
      }
    }
  ],
  "response": {
    "format": "csv"
  }
}

In [13]:
response = requests.post(URL, json=query)

In [14]:
response.status_code

200

In [15]:
response.text

'"region","kjønn","alder","yrke","Lønnstakere 2018"\r\n"0 Hele landet","Begge kjønn","15-74 år","1 Ledere",221170\r\n"0 Hele landet","Begge kjønn","15-74 år","2 Akademiske yrker",631074\r\n"0 Hele landet","Begge kjønn","15-74 år","3_01-03 Høyskole- og militære yrker",389323\r\n"0 Hele landet","Begge kjønn","15-74 år","4 Kontoryrker",169420\r\n"0 Hele landet","Begge kjønn","15-74 år","5 Salgs- og serviceyrker",571977\r\n"0 Hele landet","Begge kjønn","15-74 år","6 Bønder, fiskere mv.",21173\r\n"0 Hele landet","Begge kjønn","15-74 år","7 Håndverkere",217546\r\n"0 Hele landet","Begge kjønn","15-74 år","8 Prosess- og maskinoperatører, transportarbeidere mv.",160907\r\n"0 Hele landet","Begge kjønn","15-74 år","9 Renholdere, hjelpearbeidere mv.",133794\r\n"30 Viken","Begge kjønn","15-74 år","1 Ledere",0\r\n"30 Viken","Begge kjønn","15-74 år","2 Akademiske yrker",0\r\n"30 Viken","Begge kjønn","15-74 år","3_01-03 Høyskole- og militære yrker",0\r\n"30 Viken","Begge kjønn","15-74 år","4 Kontoryrk

## Area

In [17]:
URL = "https://data.ssb.no/api/v0/no/table/11342"

query = {
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "all",
        "values": [
          "*"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "Folkemengde",
          "ArealKm2",
          "LandArealKm2",
          "FolkeLandArealKm2"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2019"
        ]
      }
    }
  ],
  "response": {
    "format": "csv"
  }
}


In [18]:
response = requests.post(URL, json=query)
response.status_code

200

In [19]:
response.text

'"region","Befolkning per 1.1. (personer) 2018","Areal (km²) 2018","Landareal (km²) 2018","Innbyggere per km² landareal 2018"\r\n"0 Hele landet",5295619,323807,304110,17\r\n"30 Viken",0,0,0,0\r\n"01 Østfold (-2019)",295420,4187,3888,76\r\n"3001 Halden",0,0,0,0\r\n"3002 Moss",0,0,0,0\r\n"3003 Sarpsborg",0,0,0,0\r\n"3004 Fredrikstad",0,0,0,0\r\n"3005 Drammen",0,0,0,0\r\n"3006 Kongsberg",0,0,0,0\r\n"3007 Ringerike",0,0,0,0\r\n"3011 Hvaler",0,0,0,0\r\n"3012 Aremark",0,0,0,0\r\n"3013 Marker",0,0,0,0\r\n"3014 Indre Østfold",0,0,0,0\r\n"3015 Skiptvet",0,0,0,0\r\n"3016 Rakkestad",0,0,0,0\r\n"3017 Råde",0,0,0,0\r\n"3018 Våler (Viken)",0,0,0,0\r\n"3019 Vestby",0,0,0,0\r\n"3020 Nordre Follo",0,0,0,0\r\n"3021 Ås",0,0,0,0\r\n"3022 Frogn",0,0,0,0\r\n"3023 Nesodden",0,0,0,0\r\n"3024 Bærum",0,0,0,0\r\n"3025 Asker",0,0,0,0\r\n"3026 Aurskog-Høland",0,0,0,0\r\n"3027 Rælingen",0,0,0,0\r\n"3028 Enebakk",0,0,0,0\r\n"3029 Lørenskog",0,0,0,0\r\n"3030 Lillestrøm",0,0,0,0\r\n"3031 Nittedal",0,0,0,0\r\n"3032 Gje